<a href="https://colab.research.google.com/github/salisuyahaya/Cotton-Disease-Prediction-Deep-Learning/blob/master/Hausa_Sentiment_Analysis_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

### Read data

In [2]:
train = pd.read_csv('/content/train (2).csv')

test = pd.read_csv('/content/test (1).csv')

data = pd.read_csv('/content/sample_submission (2).csv')

In [3]:
train.columns

Index(['id', 'NewsText', 'label'], dtype='object')

In [4]:
train.head()

,id,NewsText,label
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1
1,1059,Karanta cikakken labarin a nan,0
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0


In [5]:
train.label.unique()

array([ 1,  0, -1])

In [6]:
train.shape

(240, 3)

In [7]:
pos = []
neg = []
neut = []

for label in train.label:
    if label == 0:
        pos.append(0)
        neg.append(0)
        neut.append(1)
    elif label == 1:
        pos.append(1)
        neg.append(0)
        neut.append(0)
    elif label == -1:
          pos.append(0)
          neg.append(1)
          neut.append(0)

In [8]:
train['Pos']= pos
train['Neg']= neg
train['Neut']= neut

In [9]:
train.head()

,id,NewsText,label,Pos,Neg,Neut
0,1553,Sannan ya ringa yin fina-finai a kan matsalol...,1,1,0,0
1,1059,Karanta cikakken labarin a nan,0,0,0,1
2,1006,Tuni dai Saudiyya ta ce za ta kaddamar da aiki...,1,1,0,0
3,1242,Wannan zai hana raguwar darajarsa kamar yadda ...,-1,0,1,0
4,1221,"Ba shi da wata iyaka, ma'ana iya shafukan da z...",0,0,0,1


### Cleaning The Data(Train)

In [10]:
def remove_punct(text1):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text1)
    return text_nopunct                                                                         

train['Text_Clean'] = train['NewsText'].apply(lambda x: remove_punct(x))

In [11]:
import nltk
nltk.download('punkt')
                                                                                                  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
from nltk import word_tokenize, WordNetLemmatizer
tokens1 = [word_tokenize(sen) for sen in train.Text_Clean]                                           

In [21]:
def lower_token(tokens1): 
    return [w.lower() for w in tokens1]    
                                                                                      
lower_tokens1 = [lower_token(token) for token in tokens1] 

In [22]:
import nltk
nltk.download('stopwords')                                                             


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.corpus import stopwords
stoplist1 = stopwords.words('english')                                                      

In [24]:
def remove_stop_words(tokens1): 
    return [word for word in tokens1 if word not in stoplist1]                              

In [25]:
filtered_words1 = [remove_stop_words(sen1) for sen1 in lower_tokens1]                       

In [27]:
result1 = [' '.join(sen1) for sen1 in filtered_words1]                                      

In [28]:
train['Text_Final1'] = result1                                                              

In [29]:
train['tokens1'] = filtered_words1                                                         

In [33]:
train = train[['Text_Final1', 'tokens1', 'label', 'Pos', 'Neg', 'Neut']]                    

In [ ]:
train[:4]                                                                                    

In [93]:
all_train_words = [word for tokens1 in train["tokens1"] for word in tokens1]
train_sentence_lengths = [len(tokens1) for tokens1 in train["tokens1"]]
TRAIN_VOCAB = sorted(list(set(all_train_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_train_words), len(TRAIN_VOCAB)))
print("Max sentence length is %s" % max(train_sentence_lengths))

4490 words total, with a vocabulary size of 1412
Max sentence length is 46


### Split data into test and train

In [35]:
#data_train, data_test = train_test_split(data, test_size=0.10, random_state=42)

###Clening The Data(Test)

In [57]:
test.head()

,id,NewsText,Text_Clean2
0,1278,"Mene ne sakamakon zaben?""Hukumar zabe ta ayyan...",Mene ne sakamakon zabenHukumar zabe ta ayyana ...
1,1721,Gwamnatin Najeriya ta ce an sako yaran ne lami...,Gwamnatin Najeriya ta ce an sako yaran ne lami...
2,1180,"""An biya kuɗin fansa kafin a sake su, amma ba ...",An biya kuɗin fansa kafin a sake su amma ba za...
3,1009,Rahotanni sun ce an shafe tsawon sa’a takwas a...,Rahotanni sun ce an shafe tsawon sa’a takwas a...
4,1588,Ya karbi lambobin yabo a gida Najeriya da kuma...,Ya karbi lambobin yabo a gida Najeriya da kuma...


In [58]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           60 non-null     int64 
 1   NewsText     60 non-null     object
 2   Text_Clean2  60 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


In [60]:
test.shape

(60, 3)

In [63]:
def remove_punct(text2):
    text_nopunct2 = ''
    text_nopunct2 = re.sub('['+string.punctuation+']', '', text2)
    return text_nopunct2                                                                         

test['Text_Clean2'] = test['NewsText'].apply(lambda x: remove_punct(x))

In [64]:
import nltk
nltk.download('punkt')
                                                                                                  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
from nltk import word_tokenize, WordNetLemmatizer
tokens2 = [word_tokenize(sen2) for sen2 in test.Text_Clean2]                                           

In [69]:
def lower_token(tokens2): 
    return [w.lower() for w in tokens2]    
                                                                                      
lower_tokens2 = [lower_token(token) for token in tokens2] 

In [70]:
import nltk
nltk.download('stopwords')                                                             


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [71]:
from nltk.corpus import stopwords
stoplist2 = stopwords.words('english')                                                      

In [72]:
def remove_stop_words(tokens2): 
    return [word for word in tokens2 if word not in stoplist2]                              

In [73]:
filtered_words2 = [remove_stop_words(sen2) for sen2 in lower_tokens2]                       

In [74]:
result2 = [' '.join(sen2) for sen2 in filtered_words2]                                      

In [75]:
test['Text_Final2'] = result2                                                              

In [76]:
test['tokens2'] = filtered_words2                                                         

In [79]:
#test = test[['Text_Final', 'tokens', 'label', 'Pos', 'Neg', 'Neut']]                    

In [80]:
test[:4]

,id,NewsText,Text_Clean2,Text_Final2,tokens2
0,1278,"Mene ne sakamakon zaben?""Hukumar zabe ta ayyan...",Mene ne sakamakon zabenHukumar zabe ta ayyana ...,mene ne sakamakon zabenhukumar zabe ta ayyana ...,"[mene, ne, sakamakon, zabenhukumar, zabe, ta, ..."
1,1721,Gwamnatin Najeriya ta ce an sako yaran ne lami...,Gwamnatin Najeriya ta ce an sako yaran ne lami...,gwamnatin najeriya ta ce sako yaran ne lami la...,"[gwamnatin, najeriya, ta, ce, sako, yaran, ne,..."
2,1180,"""An biya kuɗin fansa kafin a sake su, amma ba ...",An biya kuɗin fansa kafin a sake su amma ba za...,biya kuɗin fansa kafin sake su amma ba zan fad...,"[biya, kuɗin, fansa, kafin, sake, su, amma, ba..."
3,1009,Rahotanni sun ce an shafe tsawon sa’a takwas a...,Rahotanni sun ce an shafe tsawon sa’a takwas a...,rahotanni sun ce shafe tsawon sa ’ takwas ana ...,"[rahotanni, sun, ce, shafe, tsawon, sa, ’, tak..."


In [81]:
all_test_words = [word for tokens2 in test["tokens2"] for word in tokens2]
test_sentence_lengths = [len(tokens2) for tokens2 in test["tokens2"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

1133 words total, with a vocabulary size of 586
Max sentence length is 44


### Load Google News Word2Vec model

In [85]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'


In [89]:
import gzip


In [90]:
!pip install wget

import wget
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

f_in = gzip.open('GoogleNews-vectors-negative300.bin.gz', 'rb')
f_out = open('GoogleNews-vectors-negative300.bin', 'wb')
f_out.writelines(f_in)

import gensim
from gensim.models import Word2Vec, KeyedVectors
from sklearn.decomposition import PCA

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, limit=100000)


In [97]:
from gensim import models
#w = models.KeyedVectors.load_word2vec_format('model.bin', binary=True)


In [98]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens1'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

### Get Embeddings

In [99]:
train_embeddings = get_word2vec_embeddings(word2vec, train, generate_missing=True)

NameError: ignored

In [100]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [101]:
tokenizer = Tokenizer(num_words=len(TRAIN_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(train["Text_Final1"].tolist())
training_sequences = tokenizer.texts_to_sequences(train["Text_Final1"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 1412 unique tokens.


In [102]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [103]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

NameError: ignored

In [104]:
test_sequences = tokenizer.texts_to_sequences(test["Text_Final2"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define CNN

In [105]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [107]:
label_names = ['Pos', 'Neg', 'Neut']

In [108]:
y_train = train[label_names].values

In [109]:
x_train = train_cnn_data
y_tr = y_train

In [110]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 50, 300)      423900      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 49, 200)      120200      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 48, 200)      180200      ['embedding[0][0]']              
                                                                                              

### Train CNN

In [122]:
num_epochs = 50
batch_size = 34

In [123]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/50
7/7 [==============================] - 0s 20ms/step - loss: 0.6727 - acc: 0.3796 - val_loss: 0.6732 - val_acc: 0.2500
Epoch 2/50
7/7 [==============================] - 0s 16ms/step - loss: 0.6719 - acc: 0.3796 - val_loss: 0.6725 - val_acc: 0.2500
Epoch 3/50
7/7 [==============================] - 0s 14ms/step - loss: 0.6712 - acc: 0.3796 - val_loss: 0.6718 - val_acc: 0.2500
Epoch 4/50
7/7 [==============================] - 0s 15ms/step - loss: 0.6705 - acc: 0.3796 - val_loss: 0.6712 - val_acc: 0.2500
Epoch 5/50
7/7 [==============================] - 0s 12ms/step - loss: 0.6698 - acc: 0.3796 - val_loss: 0.6706 - val_acc: 0.2500
Epoch 6/50
7/7 [==============================] - 0s 13ms/step - loss: 0.6691 - acc: 0.3796 - val_loss: 0.6699 - val_acc: 0.2500
Epoch 7/50
7/7 [==============================] - 0s 12ms/step - loss: 0.6685 - acc: 0.3796 - val_loss: 0.6693 - val_acc: 0.2500
Epoch 8/50
7/7 [==============================] - 0s 11ms/step - loss: 0.6678 - acc: 0.3796 - val

### Test CNN

In [115]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

1/1 [==============================] - 1s 693ms/step


In [116]:
labels = [1, 0]

In [117]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

In [119]:
#sum(test.label==prediction_labels)/len(prediction_labels)

In [121]:
#test.label.value_counts()